In [9]:
import micrograd
import centigrad
import time
from sklearn.datasets import make_moons

# Generate dataset

In [10]:
samples = 100
X, y = make_moons(n_samples=samples, noise=0.1, random_state=42)
y = y.reshape((samples, 1))

# Centigrad

In [13]:
from centigrad.network import Network
from centigrad.variable import Variable

In [14]:
X_centigrad = X.T
y_centigrad = y.T

In [15]:
start = time.time()
nn_centigrad = Network(2, [16, 16, 1], ["relu", "relu", "linear"])
nn_centigrad.fit(X_centigrad, y_centigrad, epochs = 100, learning_rate = 0.01)
end = time.time()
print(f"Initial Loss: {nn_centigrad.epoch_losses[0]}")
print(f"Final Loss: {nn_centigrad.epoch_losses[-1]}")
print("-"*50)
print(f"Time elapsed: {(end-start)/60:.2f} minutes")

100%|███████████████████████████████████████████████████████████████████████████████████████| 100/100 [04:20<00:00,  2.61s/it]

Initial Loss: 62.88547507719301
Final Loss: 0.09267759186477402
--------------------------------------------------
Time elapsed: 4.35 minutes


# Micrograd

In [11]:
from micrograd.engine import Value
from micrograd.nn import Neuron, Layer, MLP
from tqdm import tqdm

In [12]:
start = time.time()
nn_micrograd = MLP(2, [16, 16, 1])

for k in tqdm(range(100)):
    # forward pass
    y_pred = [nn_micrograd(x) for x in X]
    loss = sum((yout - ygt)**2 for ygt, yout in zip(y, y_pred))
    if k==0:
        print(f"Initial Loss: {loss}")
    nn_micrograd.zero_grad()
    loss.backward()
    
    #update
    for p in nn_micrograd.parameters():
        p.data -= 0.01*p.grad
        
loss = sum((yout - ygt)**2 for ygt, yout in zip(y, y_pred))    
end = time.time()
print(f"Final Loss: {loss}")
print("-"*50)
print(f"Time elapsed: {(end-start)/60:.2f} minutes")

  1%|▉                                                                                        | 1/100 [00:00<01:01,  1.60it/s]

Initial Loss: Value(data=[355.38703203], grad=0)


100%|███████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:58<00:00,  1.71it/s]

Final Loss: Value(data=[9.40458638e+33], grad=0)
--------------------------------------------------
Time elapsed: 0.97 minutes
